<style>
    h1 {
    font-family: 'Segoe UI Semibold';
    font-size: 40px;
    text-align: center;
  }
</style>

# Proyecto 3: Saber 11

<style>
  .image {
            overflow: hidden;
            width: 100%;
            border-radius: 20px;
            object-fit: cover;
        }
  .container {
            width: 100%;
            height: 300px;
        }

</style>

<div class="container">
    <img src="../img/image.png" class="image"/>
</div>

<style>
  p {
    font-family: 'Segoe UI Light';
    font-size: 20px;
    text-align: center;
  }
</style>

<div style="text-align: center;">
  Analítica Computacional para la Toma de Decisiones IIND-4130 <br>
  Universidad de los Andes &copy; 2024 <br>
  19 de noviembre de 2024 <br><br>
  <strong>GRUPO 7:</strong> <br>
  Laura Calderón - 202122045 <br>
  Camilo Duque - 202024289<br>
  Daniela Espinosa - 202022615<br>
</div>


## **Tarea 2**
Limpieza de Datos

### **Proceso de Limpieza Previo en AWS Athena**

A continuación, se describe el proceso de limpieza y preparación realizado en AWS Athena para la obtención de una tabla depurada que será utilizada en el análisis:

1. **Filtrado por Departamento:**
   - Se seleccionaron únicamente los registros correspondientes a colegios ubicados en el departamento de **Boyacá**, filtrando la tabla según el campo `cole_depto_ubicacion`.

2. **Eliminación de Registros Duplicados:**
   - Se utilizó la cláusula `DISTINCT` para eliminar registros duplicados de la tabla, asegurando que cada fila representara datos únicos.

3. **Manejo de Datos Faltantes:**
   - Se observó que para los datos correspondientes a los periodos entre `20101` y `20141`, no se disponía de registros sobre los puntajes obtenidos por los estudiantes en las pruebas ICFES. Esto representaba un **43% de datos faltantes** en las columnas relacionadas con los puntajes.
   - Dado que estos registros no aportaban valor al análisis, se decidió **eliminarlos**. Por lo tanto, el estudio se enfoca únicamente en datos desde el periodo `20142` en adelante.

4. **Eliminación de Columnas Redundantes o Poco Informativas:**
   - **`cole_cod_depto_ubicacion`:** Representa únicamente el identificador del departamento (15 para Boyacá). Dado que todos los registros pertenecen al mismo departamento, esta columna se eliminó.
   - **`estu_consecutivo`:** Se verificó que este campo, utilizado como identificador único del estudiante, no tuviera valores duplicados. Tras confirmar su unicidad, se eliminó al no ser relevante para el análisis.
   - **`cole_depto_ubicacion`:** Esta columna también se eliminó, ya que toda la tabla corresponde al departamento de Boyacá y, por ende, no añade información adicional.
   - **`cole_cod_mcpio_ubicacion`:** Este campo numérico representaba el municipio del colegio. Sin embargo, su información es redundante con la columna `cole_mcpio_ubicacion`, que contiene el nombre del municipio. Por lo tanto, se eliminó.
   - **`estu_cod_depto_presentacion` y `estu_cod_mcpio_presentacion`:** Ambas columnas contienen códigos numéricos que representan los departamentos y municipios de presentación del estudiante, pero la misma información ya está disponible en `estu_depto_presentacion` y `estu_mcpio_presentacion`. Por esta razón, se eliminaron.
   - **`estu_cod_reside_depto` y `estu_cod_reside_mcpio`:** Similar a los casos anteriores, estas columnas contienen códigos de los departamentos y municipios de residencia, pero la misma información ya está en `estu_depto_reside` y `estu_mcpio_reside`. Se procedió a eliminarlas.
   - **`cole_cod_dane_establecimiento` y `cole_cod_dane_sede`:** Estas columnas contienen identificadores únicos de los colegios y sedes, pero no aportan valor al análisis y, por ende, se eliminaron.

5. **Resultado Final:**
   - El resultado del proceso es una tabla depurada en AWS Athena. Esta tabla se exportó en formato CSV y será utilizada para continuar con el proceso de limpieza y análisis en este notebook.


<style>
  .image {
            overflow: hidden;
            width: 100%;
            border-radius: 20px;
            object-fit: cover;
        }
  .container {
            width: 100%;
            height: 750px;
        }

</style>

<div class="container">
    <img src="../img/Consulta_Final.png" class="image"/>
</div>

In [91]:
import pandas as pd
dta = pd.read_csv('../data/Datos_Boyaca.csv')

# Mostrar información básica del DataFrame
print(f"Total de registros: {dta.shape[0]}")
print(f"Total de columnas: {dta.shape[1]}")

# Mostrar un resumen de las columnas
print("\nResumen de columnas:")
print(dta.info())



Total de registros: 100049
Total de columnas: 41

Resumen de columnas:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100049 entries, 0 to 100048
Data columns (total 41 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   periodo                      100049 non-null  int64  
 1   estu_tipodocumento           100049 non-null  object 
 2   cole_area_ubicacion          100049 non-null  object 
 3   cole_bilingue                91518 non-null   object 
 4   cole_calendario              100049 non-null  object 
 5   cole_caracter                99600 non-null   object 
 6   cole_codigo_icfes            100049 non-null  int64  
 7   cole_genero                  100049 non-null  object 
 8   cole_jornada                 100049 non-null  object 
 9   cole_mcpio_ubicacion         100049 non-null  object 
 10  cole_naturaleza              100049 non-null  object 
 11  cole_nombre_establecimiento  100049 non-null  

In [92]:
# Verificar que no haya filas duplicadas
duplicados = dta.duplicated()

# Contar cuántas filas duplicadas hay
num_duplicados = duplicados.sum()
print(f"Número de filas duplicadas: {num_duplicados}")

Número de filas duplicadas: 0


In [93]:
# Calcular la cantidad de valores faltantes y el porcentaje
faltantes = dta.isnull().sum()
porcentaje_faltantes = (dta.isnull().sum() / len(dta)) * 100

resumen_faltantes = pd.DataFrame({
    'Valores Faltantes': faltantes,
    'Porcentaje (%)': porcentaje_faltantes
})

# Filtrar para mostrar solo las columnas con valores faltantes
resumen_faltantes = resumen_faltantes[resumen_faltantes['Valores Faltantes'] > 0]
resumen_faltantes = resumen_faltantes.sort_values(by='Porcentaje (%)', ascending=False)

print(resumen_faltantes)

                      Valores Faltantes  Porcentaje (%)
cole_bilingue                      8531        8.526822
fami_estratovivienda               2642        2.640706
fami_tieneinternet                 2464        2.462793
fami_educacionmadre                2382        2.380833
fami_educacionpadre                2381        2.379834
fami_personashogar                 1598        1.597217
fami_cuartoshogar                  1541        1.540245
fami_tieneautomovil                1540        1.539246
fami_tienelavadora                 1491        1.490270
fami_tienecomputador               1489        1.488271
cole_caracter                       449        0.448780
estu_genero                         115        0.114944
estu_depto_reside                    62        0.061970
estu_mcpio_reside                    62        0.061970
desemp_ingles                         2        0.001999
punt_ingles                           2        0.001999


In [94]:
# Crear un diccionario para almacenar la proporción máxima y el valor más repetido por columna
columnas_dominantes = {}

for columna in dta.columns:
    valor_mas_comun = dta[columna].value_counts(normalize=True, dropna=False).idxmax()
    proporción_max = dta[columna].value_counts(normalize=True, dropna=False).max()
    columnas_dominantes[columna] = {'Proporción Max (%)': proporción_max * 100, 'Valor Más Común': valor_mas_comun}

resultado = pd.DataFrame.from_dict(columnas_dominantes, orient='index')
columnas_repetitivas = resultado[resultado['Proporción Max (%)'] >= 95]
columnas_repetitivas = columnas_repetitivas.sort_values(by='Proporción Max (%)', ascending=False)

print("Columnas con 80% o más de valores repetidos:")
print(columnas_repetitivas)


Columnas con 80% o más de valores repetidos:
                          Proporción Max (%) Valor Más Común
cole_calendario                   100.000000               A
estu_estudiante                   100.000000      ESTUDIANTE
estu_privado_libertad              99.947026               N
estu_estadoinvestigacion           99.917041        PUBLICAR
estu_nacionalidad                  99.909045        COLOMBIA
estu_pais_reside                   99.909045        COLOMBIA
estu_depto_reside                  99.371308          BOYACA
estu_depto_presentacion            98.460754          BOYACA
cole_genero                        97.716119           MIXTO
cole_sede_principal                97.330308               S


In [95]:
# Identificar columnas con más del 95% de valores repetidos y eliminnarlas
columnas_a_eliminar = columnas_repetitivas.index.tolist()
dta= dta.drop(columns=columnas_a_eliminar)

In [96]:
# Identificar columnas numéricas en el DataFrame
columnas_numericas = dta.select_dtypes(include=['number']).columns
print("Columnas numéricas:")
print(columnas_numericas)

if len(columnas_numericas) == 0:
    print("No hay columnas numéricas en el DataFrame.")
else:
    print(f"Total de columnas numéricas: {len(columnas_numericas)}")

Columnas numéricas:
Index(['periodo', 'cole_codigo_icfes', 'punt_ingles', 'punt_matematicas',
       'punt_sociales_ciudadanas', 'punt_c_naturales', 'punt_lectura_critica',
       'punt_global'],
      dtype='object')
Total de columnas numéricas: 8


In [97]:
# Identificar las columnas categóricas y las columnas numéricas con faltantes
columnas_categoricas = [
    'cole_bilingue', 'fami_estratovivienda', 'fami_tieneinternet', 
    'fami_educacionmadre', 'fami_educacionpadre', 'fami_tieneautomovil',
    'fami_tienelavadora', 'fami_tienecomputador', 'cole_caracter', 
    'estu_genero','fami_personashogar', 'fami_cuartoshogar','desemp_ingles','estu_cod_reside_mcpio','estu_mcpio_reside'
]

columnas_numericas = [
    'punt_ingles', 'punt_global'
]

print("\nSe imputaan valores faltante en columnas categóricas con la moda:")
# Imputar valores faltantes en columnas categóricas con la moda
for col in columnas_categoricas:
    if col in dta.columns:
        moda = dta[col].mode()[0]
        dta[col] = dta[col].fillna(moda)
        print(f" '{col}'  -  {moda}")

print("\nSe imputaan valores faltante en columnas numéricas con la media:")
# Imputar valores faltantes en columnas numéricas con la media
for col in columnas_numericas:
    if col in dta.columns:
        media = dta[col].mean()
        dta[col] = dta[col].fillna(media)
        print(f" '{col}'  -  {media}")

# Verificar que no queden valores faltantes
faltantes_restantes = dta.isnull().sum()
print("\nValores faltantes restantes por columna después de la imputación:")
print(faltantes_restantes[faltantes_restantes > 0])


Se imputaan valores faltante en columnas categóricas con la moda:
 'cole_bilingue'  -  N
 'fami_estratovivienda'  -  Estrato 2
 'fami_tieneinternet'  -  No
 'fami_educacionmadre'  -  Secundaria (Bachillerato) completa
 'fami_educacionpadre'  -  Primaria incompleta
 'fami_tieneautomovil'  -  No
 'fami_tienelavadora'  -  Si
 'fami_tienecomputador'  -  Si
 'cole_caracter'  -  TÉCNICO/ACADÉMICO
 'estu_genero'  -  F
 'fami_personashogar'  -  3 a 4
 'fami_cuartoshogar'  -  Tres
 'desemp_ingles'  -  A-
 'estu_mcpio_reside'  -  TUNJA

Se imputaan valores faltante en columnas numéricas con la media:
 'punt_ingles'  -  51.63191300088958
 'punt_global'  -  264.74132674989255

Valores faltantes restantes por columna después de la imputación:
Series([], dtype: int64)


In [100]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Seleccionar columnas no numéricas
columnas_no_numericas = dta.select_dtypes(include=['object']).columns

# Diccionario para almacenar los LabelEncoders y sus mapeos
label_encoders = {}
mapeos = {}

for col in columnas_no_numericas:
    le = LabelEncoder()
    dta[col] = le.fit_transform(dta[col].astype(str))
    label_encoders[col] = le
    mapeos[col] = dict(zip(le.classes_, le.transform(le.classes_)))


for col, mapeo in mapeos.items():
    print(f"Columna: {col}")
    for original, transformado in mapeo.items():
        print(f"  {original} -> {transformado}")
    print("-" * 30)


Columna: estu_tipodocumento
  CC -> 0
  CE -> 1
  CR -> 2
  NES -> 3
  PC -> 4
  PE -> 5
  PEP -> 6
  PPT -> 7
  RC -> 8
  TI -> 9
  V -> 10
------------------------------
Columna: cole_area_ubicacion
  RURAL -> 0
  URBANO -> 1
------------------------------
Columna: cole_bilingue
  N -> 0
  S -> 1
------------------------------
Columna: cole_caracter
  ACADÉMICO -> 0
  NO APLICA -> 1
  TÉCNICO -> 2
  TÉCNICO/ACADÉMICO -> 3
------------------------------
Columna: cole_jornada
  COMPLETA -> 0
  MAÑANA -> 1
  NOCHE -> 2
  SABATINA -> 3
  TARDE -> 4
  UNICA -> 5
------------------------------
Columna: cole_mcpio_ubicacion
  ALMEIDA -> 0
  AQUITANIA -> 1
  ARCABUCO -> 2
  BELEN -> 3
  BELÉN -> 4
  BERBEO -> 5
  BETEITIVA -> 6
  BETÉITIVA -> 7
  BOAVITA -> 8
  BOYACA -> 9
  BOYACÁ -> 10
  BRICEÑO -> 11
  BUENAVISTA -> 12
  BUSBANZA -> 13
  BUSBANZÁ -> 14
  CALDAS -> 15
  CAMPOHERMOSO -> 16
  CERINZA -> 17
  CHINAVITA -> 18
  CHIQUINQUIRA -> 19
  CHIQUINQUIRÁ -> 20
  CHIQUIZA -> 21
  CHISCAS